In [ ]:
# !pip install tensorflow
!pip install scanpy==1.9.1
!pip install matplotlib==3.6
!pip install scib
!pip install louvain

# ---------------------------------------------------------------------
# WHEN USING COLAB, PLEASE RESTART RUNTIME AFTER RUNNING THIS CELL
# ---------------------------------------------------------------------

# (some of the packages being used in this notebook are automatically loaded
# before installation of a different version and they need to be reloaded)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.7 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.4-py3-none-any.whl size=86770 sha256=bff736647fb4fc39fb87f5cd5366a30246b4fdbab9f574416a1e46f2f3b12811
  Stored in directory: /root/.cache/pip/wheels/fb/66/29/199acf5784d0f7b8add6d466175ab45506c96e386ed5dd0633
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=213e8ab29cb3c8cd28d3c3596a45d35b85fbb994675b4abb2baaa320516d5408
  Stored in directory: /root/.cache/pip/wheels/6a/aa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.4 MB/s eta 0:00:00


In [ ]:
# LISI scores use the knn_graph.o file created by the cpp code: knn_graph.cpp
# In order to use these metrics we need to recompile the code in the current
# environment, and replace the existing file with the compiled new file,
# using the following code:

!wget https://raw.githubusercontent.com/theislab/scib/main/scib/knn_graph/knn_graph.cpp
!g++ -O3 -o knn_graph.o knn_graph.cpp

import shutil
import pathlib
import scib
import os

root = pathlib.Path(scib.__file__).parent
print(root)

cpp_file_path = (
    # root / "knn_graph/knn_graph.o"
    root / "knn_graph/"
)

os.remove(str(root / "knn_graph/knn_graph.o"))
shutil.move("knn_graph.o", str(cpp_file_path))

--2023-10-05 08:32:39--  https://raw.githubusercontent.com/theislab/scib/main/scib/knn_graph/knn_graph.cpp
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14207 (14K) [text/plain]
Saving to: ‘knn_graph.cpp’

knn_graph.cpp       100%[===================>]  13.87K  --.-KB/s    in 0s      

2023-10-05 08:32:40 (113 MB/s) - ‘knn_graph.cpp’ saved [14207/14207]

/usr/local/lib/python3.10/dist-packages/scib


'/usr/local/lib/python3.10/dist-packages/scib/knn_graph/knn_graph.o'

In [ ]:
import os
from pathlib import Path

import scipy
import pandas as pd
import scanpy as sc
import numpy as np
import random
import scib as scIB
import tensorflow as tf
import warnings
import traceback
import sys

sys.path.append('/content/drive/MyDrive/modules/')
from evaluate_integration import evaluate_integration
from datasets_dict import datasets

# list of methods to evaluate
methods = [
            'scvi',
            'scanvi',
            'scanorama',
            'combat',
            'Seurat',
            'AutoClass',
            'scgen',
            'ABC',
            'harmony',
            'scDREAMER',
            'CLEAR',
            ]

embedding_methods = ['ABC', 'harmony', 'scDREAMER', 'CLEAR']
# path to the original dataset (after subset to 3000 highly variable genes)
orig_base_path = '/content/drive/MyDrive/Colab Notebooks/integrationDatasets/'

# path to the metrics csv file
metrics_path = os.path.join(orig_base_path, 'final_metrics')

# create metrics folder if does not exist
Path(metrics_path).mkdir(parents=True, exist_ok=True)


# which integration metrics to calculate (mark True to use the metric)
eval_params = {
    'silhouette_': True,
    'nmi_': True,
    'ari_': True,
    'cell_cycle_': True,    # turns to false for ATAC
    'isolated_labels_': True,
    'hvg_overlap_': True,
    'graph_conn_': True,
    'lisi_graph_': True,
    'trajectory_': True     # turns to false if pseudotime info is not present
}


# Evaluate each dataset in datasets
for dataset_name in datasets.keys():

    # get dataset parameters from the dictionary
    label_key = datasets[dataset_name]['label_key']
    batch_key = datasets[dataset_name]['batch_key']
    subsample = datasets[dataset_name]['subsample']
    atac = datasets[dataset_name]['ATAC']
    organism = datasets[dataset_name]['organism']

    if atac:
      data_type = 'ATAC'
    else:
      data_type = 'RNA'


    # load original (unintegrated) dataset
    orig_path = os.path.join(orig_base_path, f"{dataset_name}_hvg.h5ad")
    orig_data = sc.read(orig_path)

    # make sure the data matrix is not sparse
    if scipy.sparse.issparse(orig_data.X):
      print("The given adata.X matrix is sparse. Converting to dense.")
      orig_data.X = orig_data.X.todense()


    # evaluate each method for the current dataset
    for method in methods:

        # skip the dataset scDREAMER failed on
        if method == 'scDREAMER' and dataset_name == 'small_atac_peaks':
          continue

        print('-' * 50)
        print("Using dataset: ", dataset_name)
        print("Evaluating method: ", method)
        print('-' * 50)

        # Set seed for reproducibility
        seed_value = 1
        os.environ['PYTHONHASHSEED'] = str(seed_value)  # the hash seed for Python
        random.seed(seed_value)  # the seed for Python's built-in random module
        np.random.seed(seed_value)  # the seed for Numpy (which Scipy relies on)
        tf.random.set_seed(seed_value)  # the seed for TensorFlow


        # load integrated dataset
        integ_path = os.path.join(orig_base_path, "integratedDatasets",
                                  method, f"{dataset_name}_integrated.h5ad")
        integ_data = sc.read(integ_path)


        # make sure data is not sparse
        if scipy.sparse.issparse(integ_data.X):
          print("The given integ_data.X matrix is sparse. Converting to dense.")
          integ_data.X = integ_data.X.todense()


        # method specific data preparations
        if method == 'ABC':
          integ_data.obsm['X_emb'] = integ_data.X.copy()

        # fix gene names for Seurat naming convention
        if method == 'Seurat':

            # if the var names contain '_' instead of "-" - use the next lines to convert
            integ_data.var_names = [id.replace('-', '_') for id in integ_data.var_names]
            orig_data.var_names = [id.replace('-', '_') for id in orig_data.var_names]

            # only include cells from Seurat integration
            orig_data = orig_data[:, integ_data.var_names]



        # --- Evaluate integration ---

        # define output type
        if method in embedding_methods:
          output_type = "embed"
        else:
          output_type = "full"

        # calculate integration scores
        print("calculating integration scores:")
        integ_scores = pd.DataFrame()

        try:
            integ_scores = evaluate_integration(orig_data, integ_data,
                                                eval_params,
                                                batch_key=batch_key,
                                                label_key=label_key,
                                                data_type=data_type,
                                                organism=organism,
                                                bio_con_w=0.5,
                                                output_type=output_type)
        except Exception as e:
            print(f'---exception when evaluating integration of {dataset_name} for {method}!')
            print(e)
            continue

        print("integration scores:")
        print(integ_scores)


        # --- save scores ---
        # Transpose the DataFrame and drop Nan columns
        integ_scores = integ_scores.T
        integ_scores = integ_scores.dropna(axis=1, how='all')

        # Rename the index of the transposed DataFrame with the value of the method
        if method in embedding_methods:
          integ_scores.index = [f"{method}_embed"]
        else:
          integ_scores.index = [method]

        # define metrics output path
        metrics_file = os.path.join(metrics_path, f'{dataset_name}_metrics.csv')

        # save the scores dataframe to a csv file
        if os.path.exists(metrics_file):
            integ_scores.to_csv(metrics_file, mode='a', index=True, header=False)
        else:
            integ_scores.to_csv(metrics_file, index=True)


<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen imp

--------------------------------------------------
Using dataset:  small_atac_windows
Evaluating method:  CLEAR
--------------------------------------------------
calculating integration scores:
PCA


<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


Nearest Neigbours


<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen imp

...

/usr/local/lib/python3.10/dist-packages/scib/metrics/lisi.py:175: UserWarning: 'batch_key' is deprecated and will be ignore
  warnings.warn("'batch_key' is deprecated and will be ignore", stacklevel=1)


using precomputed kNN graph
Convert nearest neighbor matrix and distances for LISI.
Compute knn on shortest paths
call /usr/local/lib/python3.10/dist-packages/scib/knn_graph/knn_graph.o /tmp/lisi_tetotp3w/graph_lisi_input.mtx /tmp/lisi_tetotp3w/graph_lisi 90 1 50
LISI score estimation


<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()


iLISI score...
using precomputed kNN graph
Convert nearest neighbor matrix and distances for LISI.
Compute knn on shortest paths
call /usr/local/lib/python3.10/dist-packages/scib/knn_graph/knn_graph.o /tmp/lisi_e_xywl9p/graph_lisi_input.mtx /tmp/lisi_e_xywl9p/graph_lisi 90 1 50
LISI score estimation
integration scores:
                                  0
NMI_cluster/label          0.526927
ARI_cluster/label          0.519626
ASW_label                  0.535041
ASW_label/batch            0.888107
PCR_batch                       NaN
cell_cycle_conservation         NaN
isolated_label_F1          0.790316
isolated_label_silhouette  0.537665
graph_conn                 0.945514
kBET                            NaN
iLISI                      0.535249
cLISI                      0.930642
hvg_overlap                     NaN
trajectory                      NaN
Bio_Conservation_avg       0.640036
Batch_Correction_avg       0.789623
Final_Score                0.714830


<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _AltairImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen imp